Este Dataset reune la info de musicales_limpio y añade/actualiza con la de musicales_listado

In [62]:
import pandas as pd



In [63]:
df_base = pd.read_csv("musicales_limpio.csv")
df_excel = pd.read_excel("musicales_listado.xlsx")

In [64]:
def normaliza_columnas(df):
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()        # quita espacios al inicio y al final
          .str.lower()        # pasa todo a minúsculas
          .str.replace(" ", "_")  # espacios -> guiones bajos
    )
    return df

df_base = normaliza_columnas(df_base)
df_excel = normaliza_columnas(df_excel)



In [65]:


def normaliza_nombres(df):
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(" ", "_")
          .str.replace("__", "_")
    )
    return df

# 1) Normalizar NOMBRES justo después de cargar
df_base  = normaliza_nombres(df_base)
df_excel = normaliza_nombres(df_excel)

# 2) Crear clave compuesta
def crear_clave(df):
    return (
        df["obra"].astype(str).str.strip().str.lower() + " | " +
        df["productora"].astype(str).str.strip().str.lower() + " | " +
        df["anio_inicio"].astype(str).str.strip() + " | " +
        df["anio_fin"].astype(str).str.strip()
    )

df_base["clave_prod"]  = crear_clave(df_base)
df_excel["clave_prod"] = crear_clave(df_excel)

base_idx  = df_base.set_index("clave_prod")
excel_idx = df_excel.set_index("clave_prod")

base_idx.update(excel_idx)

claves_nuevas = excel_idx.index.difference(base_idx.index)
df_nuevos = excel_idx.loc[claves_nuevas]

df_final = pd.concat([base_idx, df_nuevos]).reset_index()



In [66]:
df = df_final.copy()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   clave_prod        75 non-null     object 
 1   obra              75 non-null     object 
 2   productora        75 non-null     object 
 3   anio_inicio       75 non-null     int64  
 4   anio_fin          62 non-null     float64
 5   teatro            75 non-null     object 
 6   ciudad_principal  75 non-null     object 
 7   gira              75 non-null     object 
 8   fuente_url        75 non-null     object 
 9   activa            75 non-null     object 
 10  duracion          62 non-null     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 6.6+ KB


In [67]:
df.head(20)


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
0,"101 dálmatas, el musical | teatropolis (gran t...","101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://www.granteatrocaixabankprincipepio.com/,True,NaN
1,aladdín | stage entertainment | 2023 | 2025.0,Aladdín,Stage Entertainment,2023,2025.0,Teatro Coliseum,Madrid,No,https://www.stage.es/musicales/aladdin/,False,2.0
2,anastasia | stage entertainment | 2018 | 2020.0,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,Madrid,No,https://www.stage.es/musicales/anastasia/,False,2.0
3,avenue q | teatropolis (gran teatro caixabank ...,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://es.wikipedia.org/wiki/Avenue_Q,True,NaN
4,billy elliot | som produce | 2017 | 2020.0,Billy Elliot,SOM Produce,2017,2020.0,Nuevo Teatro Alcalá,Madrid,Sí,https://somproduce.com/musicales/billy-elliot/,False,3.0
5,cabaret | let's go company | 2025 | nan,Cabaret,Let's Go Company,2025,NaN,Albéniz,Madrid,No,fuente: observación personal,True,NaN
6,cenicienta | stage entertainment | 2025 | nan,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,Madrid,No,fuente: observación personal,True,NaN
7,charlie y la fábrica de chocolate | let's go c...,Charlie y la fábrica de chocolate,Let's Go Company,2021,2022.0,Espacio Ibercaja Delicias,Madrid,Sí,https://www.teatroalamedia.com/charlie-y-la-fa...,False,1.0
8,chicago | som produce | 2023 | 2025.0,Chicago,SOM Produce,2023,2025.0,Teatro Apolo,Madrid,Sí,https://somproduce.com/musicales/chicago/,False,2.0
9,dirty dancing | let's go company | 2018 | 2023.0,Dirty Dancing,Let's Go Company,2018,2023.0,Teatro Nuevo Alcalá,Madrid,Sí,https://www.entradas.com/artist/dirty-dancing/,False,5.0


In [68]:
df.duplicated(subset=["obra", "productora", "anio_inicio", "anio_fin"]).sum()


np.int64(0)

In [69]:
df[["obra", "productora", "anio_inicio", "anio_fin"]].isna().sum()


obra            0
productora      0
anio_inicio     0
anio_fin       13
dtype: int64

In [70]:
df[df["anio_fin"].isna()][["obra", "productora", "anio_inicio", "anio_fin"]]


,obra,productora,anio_inicio,anio_fin
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN
3,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN
5,Cabaret,Let's Go Company,2025,NaN
6,Cenicienta,Stage Entertainment,2025,NaN
11,El Fantasma de la Ópera,Let's Go Company,2023,NaN
15,El Rey León,Stage Entertainment,2011,NaN
19,Houdini,Let's Go Company,2025,NaN
26,La Llamada,Suma Latina / Los Javis,2013,NaN
27,Los Miserables,SOM Produce,2025,NaN
28,Los Pilares de la Tierra,Beon Entertainment,2024,NaN


In [71]:
mask_hair = (
    df["obra"].astype(str).str.strip().str.lower() == "hair"
) & (
    df["productora"].astype(str).str.lower().str.contains("william morris")
) & (
    df["teatro"].astype(str).str.lower().str.contains("apolo")
)

# Poner año fin 2011
df.loc[mask_hair, "anio_fin"] = 2011

# Cambiar gira a "No"
df.loc[mask_hair, "gira"] = "No"


In [72]:
df.loc[
    df["obra"].str.lower().str.strip() == "hair",
    ["obra", "productora", "teatro", "anio_inicio", "anio_fin", "gira"]
]


,obra,productora,teatro,anio_inicio,anio_fin,gira
58,Hair,The William Morris Agency Endeavor Entertainme...,Apolo,2010,2011.0,No


In [73]:
df.loc[
    df["obra"].str.strip() == "La llamada",
    "anio_fin"
] = 2024


In [74]:
df.loc[
    df["obra"].str.strip().str.lower() == "la llamada",
    ["obra", "anio_inicio", "anio_fin"]
]


,obra,anio_inicio,anio_fin
26,La Llamada,2013,NaN
60,La llamada,2013,2024.0


In [75]:
mask_llamada = (
    df["obra"].str.strip().str.lower() == "la llamada"
) & (
    df["anio_inicio"] == 2013
)


In [76]:
df.loc[mask_llamada, "anio_fin"] = 2024


In [77]:
df[df["obra"].str.strip().str.lower() == "la llamada"]


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
26,la llamada | suma latina / los javis | 2013 | nan,La Llamada,Suma Latina / Los Javis,2013,2024.0,Teatro Lara,Madrid,Sí,https://es.wikipedia.org/wiki/La_llamada_(musi...,True,NaN
60,la llamada | los javies | 2013 | 2024,La llamada,Los javies,2013,2024.0,Lara,Madrid,Si,La llamada (2013) - Cartelera Musicales,False,95.0


In [78]:
# Eliminar la fila 26 y reindexar
df = df.drop(index=26).reset_index(drop=True)


In [79]:
df.loc[df["obra"].str.strip().str.lower() == "la llamada",
       ["obra", "anio_inicio", "anio_fin"]]


,obra,anio_inicio,anio_fin
59,La llamada,2013,2024.0


In [80]:
mask_llamada = df["obra"].str.strip().str.lower() == "la llamada"
df.loc[mask_llamada, "productora"] = "Los Javis"


In [81]:
df.loc[mask_llamada, ["obra", "productora"]]


,obra,productora
59,La llamada,Los Javis


In [82]:
df[df["anio_fin"].isna()][["obra", "productora", "anio_inicio", "anio_fin"]]

,obra,productora,anio_inicio,anio_fin
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN
3,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN
5,Cabaret,Let's Go Company,2025,NaN
6,Cenicienta,Stage Entertainment,2025,NaN
11,El Fantasma de la Ópera,Let's Go Company,2023,NaN
15,El Rey León,Stage Entertainment,2011,NaN
19,Houdini,Let's Go Company,2025,NaN
26,Los Miserables,SOM Produce,2025,NaN
27,Los Pilares de la Tierra,Beon Entertainment,2024,NaN
29,Mamma Mia!,SOM Produce,2022,NaN


In [88]:
df["duracion"] = pd.to_numeric(df["duracion"], errors="coerce")


In [89]:
df["duracion"] = pd.to_numeric(df["duracion"], errors="coerce")


In [90]:
mask_ok = df["duracion"].between(40, 240)


In [91]:
df.loc[~mask_ok, "duracion"] = pd.NA


In [92]:
df[df["duracion"].isna()][["obra", "productora", "anio_inicio", "duracion"]]


,obra,productora,anio_inicio,duracion
0,"101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN
1,Aladdín,Stage Entertainment,2023,NaN
2,Anastasia,Stage Entertainment,2018,NaN
3,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN
4,Billy Elliot,SOM Produce,2017,NaN
5,Cabaret,Let's Go Company,2025,NaN
6,Cenicienta,Stage Entertainment,2025,NaN
7,Charlie y la fábrica de chocolate,Let's Go Company,2021,NaN
8,Chicago,SOM Produce,2023,NaN
9,Dirty Dancing,Let's Go Company,2018,NaN


In [93]:
nombres_0_15 = [
    "101 Dálmatas, el musical",
    "Aladdin",
    "Anastasia",
    "Avenue Q",
    "Billy Elliot",
    "Cabaret",
    "Cenicienta",
    "Charlie y la fábrica de chocolate",
    "Chicago",
    "Dirty Dancing",
    "El Día de la Marmota",
    "El Fantasma de la Ópera",
    "El Guardaspaldas",
    "El Jovencito Frankenstein",
    "El Médico",
    "El Rey León"
]

df[df["obra"].isin(nombres_0_15) & df["duracion"].isna()]


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
0,"101 dálmatas, el musical | teatropolis (gran t...","101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://www.granteatrocaixabankprincipepio.com/,True,NaN
2,anastasia | stage entertainment | 2018 | 2020.0,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,Madrid,No,https://www.stage.es/musicales/anastasia/,False,NaN
3,avenue q | teatropolis (gran teatro caixabank ...,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://es.wikipedia.org/wiki/Avenue_Q,True,NaN
4,billy elliot | som produce | 2017 | 2020.0,Billy Elliot,SOM Produce,2017,2020.0,Nuevo Teatro Alcalá,Madrid,Sí,https://somproduce.com/musicales/billy-elliot/,False,NaN
5,cabaret | let's go company | 2025 | nan,Cabaret,Let's Go Company,2025,NaN,Albéniz,Madrid,No,fuente: observación personal,True,NaN
6,cenicienta | stage entertainment | 2025 | nan,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,Madrid,No,fuente: observación personal,True,NaN
7,charlie y la fábrica de chocolate | let's go c...,Charlie y la fábrica de chocolate,Let's Go Company,2021,2022.0,Espacio Ibercaja Delicias,Madrid,Sí,https://www.teatroalamedia.com/charlie-y-la-fa...,False,NaN
8,chicago | som produce | 2023 | 2025.0,Chicago,SOM Produce,2023,2025.0,Teatro Apolo,Madrid,Sí,https://somproduce.com/musicales/chicago/,False,NaN
9,dirty dancing | let's go company | 2018 | 2023.0,Dirty Dancing,Let's Go Company,2018,2023.0,Teatro Nuevo Alcalá,Madrid,Sí,https://www.entradas.com/artist/dirty-dancing/,False,NaN
10,el día de la marmota | nostromo live | 2024 | ...,El Día de la Marmota,Nostromo Live,2024,2025.0,Teatro Coliseum (BCN),Barcelona,No,https://www.nostromolive.com/espectaculos/el-d...,False,NaN


In [96]:
duraciones_oficiales = {
    "101 Dálmatas, el musical": 85,
    "Aladdín": 145,
    "Anastasia": 145,
    "Avenue Q": 100,
    "Billy Elliot": 150,
    "Cabaret": 150,
    "Cenicienta": 145,
    "Charlie y la fábrica de chocolate": 150,
    "Chicago": 150,
    "Dirty Dancing": 145,
    "El Día de la Marmota": 150,
    "El Fantasma de la Ópera": 150,
    "El Guardaspaldas": 125,
    "El Jovencito Frankenstein": 135,
    "El Médico": 170,
    "El Rey León": 165,
}

# Rellena solo donde está en NaN o vacío
df["duracion"] = df["duracion"].fillna(
    df["obra"].map(duraciones_oficiales)
)


In [97]:
df[df["duracion"].isna()][["obra", "productora", "anio_inicio", "duracion"]]

,obra,productora,anio_inicio,duracion
12,El Guardaespaldas,Stage Entertainment,2017,NaN
16,El Tiempo entre Costuras,Beon Entertainment,2021,NaN
17,"Ghost, el musical",Let's Go Company,2019,NaN
18,Grease,SOM Produce,2021,NaN
19,Houdini,Let's Go Company,2025,NaN
20,Kinky Boots,Let's Go Company,2021,NaN
21,Kinky Boots,Theatre Properties,2024,NaN
22,La Alegría que Pasa,Dagoll Dagom,2022,NaN
23,La Familia Addams,Let's Go Company,2017,NaN
24,La Historia Interminable,Beon Entertainment,2022,NaN


In [98]:
# Diccionario: título -> duración en minutos
dur_map = {
    "Aladdín": 145,
    "El Guardaespaldas": 160,
    "El Tiempo entre Costuras": 150,
    "Ghost, el musical": 150,
    "Grease": 150,
    "Houdini": 95,
    "Kinky Boots": 140,              # se aplicará a las dos filas con Kinky Boots que tengan NaN
    "La Alegría que Pasa": 105,
    "La Familia Addams": 150,
    "La Jaula de las Locas": 150,
    "Los Miserables": 170,
    "Los Pilares de la Tierra": 145,
    "Los Productores": 150,
    "Mamma Mia!": 155,
    "Matilda": 120,
    "Priscilla, reina del desierto": 150,
    "Romeo y Julieta, el musical": 130,
    "School of Rock": 160,
    "Sister Act": 120,
    "The Book of Mormon": 140,
    "The Full Monty, el musical": 150,
    "The Hole": 120,
    "Tina": 160,
    "We Will Rock You": 150,
    "Mar i Cel": 150,
}

# Rellenar solo donde duracion es NaN y el título coincide
for obra, mins in dur_map.items():
    mask = (
        df["obra"].str.strip().str.lower() == obra.lower()
    ) & df["duracion"].isna()
    df.loc[mask, "duracion"] = mins

# Comprobar que ya no quedan NaN en esos títulos
df[df["obra"].isin(dur_map.keys())][["obra", "anio_inicio", "duracion"]]


,obra,anio_inicio,duracion
1,Aladdín,2023,145.0
12,El Guardaespaldas,2017,160.0
16,El Tiempo entre Costuras,2021,150.0
17,"Ghost, el musical",2019,150.0
18,Grease,2021,150.0
19,Houdini,2025,95.0
20,Kinky Boots,2021,140.0
21,Kinky Boots,2024,140.0
22,La Alegría que Pasa,2022,105.0
23,La Familia Addams,2017,150.0


In [99]:
df["duracion"].isna().sum()


np.int64(2)

In [105]:
df[df["duracion"].isna()]


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
24,la historia interminable | beon entertainment ...,La Historia Interminable,Beon Entertainment,2022,2024.0,Teatro Calderón,Madrid,Sí,https://www.beonworldwide.com/proyectos/la-his...,False,NaN
40,mari i cel | dagoll dagom | 2024 | 2025.0,Mari i Cel,Dagoll Dagom,2024,2025.0,Teatre Victòria,Barcelona,Sí,https://maricelelmusical.cat/,False,NaN


In [110]:
# La Historia Interminable (Beón)
df.loc[
    df["obra"].str.strip().str.lower() == "la historia interminable",
    "duracion"
] = 160

# Mar i Cel (Dagoll Dagom)
df.loc[
    df["obra"].str.strip().str.lower() == "mari i cel",
    "duracion"
] = 150


In [111]:
df["duracion"].isna().sum()

np.int64(0)

In [112]:
df.duplicated(subset=["obra", "productora"]).sum()


np.int64(1)

In [114]:
df[df["obra"].str.strip().str.lower() == "sister act"]


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
34,sister act | stage entertainment | 2015 | 2015.0,Sister Act,Stage Entertainment,2015,2015.0,Teatro Nuevo Alcalá,Madrid,Sí,https://es.wikipedia.org/wiki/Sister_Act_(musi...,False,120.0
69,sister act | stage entertainment | 2014 | 2016,Sister Act,Stage Entertainment,2014,2016.0,Tívoli,Barcelona,Si,https://www.stage.es/musicales/sister-act,False,150.0


In [115]:
df = df.drop(index=34)


In [116]:
df[df["obra"].str.strip().str.lower() == "sister act"]


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
69,sister act | stage entertainment | 2014 | 2016,Sister Act,Stage Entertainment,2014,2016.0,Tívoli,Barcelona,Si,https://www.stage.es/musicales/sister-act,False,150.0


In [117]:
df.info()
df["anio_fin"].isna().sum()
df["activa"].value_counts()


<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 0 to 73
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   clave_prod        73 non-null     object 
 1   obra              73 non-null     object 
 2   productora        73 non-null     object 
 3   anio_inicio       73 non-null     int64  
 4   anio_fin          61 non-null     float64
 5   teatro            73 non-null     object 
 6   ciudad_principal  73 non-null     object 
 7   gira              73 non-null     object 
 8   fuente_url        73 non-null     object 
 9   activa            73 non-null     object 
 10  duracion          73 non-null     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 6.8+ KB


activa
False    32
False    28
True     12
FaLse     1
Name: count, dtype: int64

In [118]:
df["duracion"].describe()


count     73.000000
mean     140.958904
std       24.687738
min       70.000000
25%      130.000000
50%      150.000000
75%      155.000000
max      180.000000
Name: duracion, dtype: float64

In [119]:
# Normalizar texto a minúsculas sin espacios
df["activa"] = (
    df["activa"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"true": True, "false": False})
)

# Comprobar que ya es booleano
df["activa"].value_counts()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 0 to 73
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   clave_prod        73 non-null     object 
 1   obra              73 non-null     object 
 2   productora        73 non-null     object 
 3   anio_inicio       73 non-null     int64  
 4   anio_fin          61 non-null     float64
 5   teatro            73 non-null     object 
 6   ciudad_principal  73 non-null     object 
 7   gira              73 non-null     object 
 8   fuente_url        73 non-null     object 
 9   activa            73 non-null     bool   
 10  duracion          73 non-null     float64
dtypes: bool(1), float64(2), int64(1), object(7)
memory usage: 6.3+ KB


In [120]:
df.groupby("activa")["obra"].count()


activa
False    61
True     12
Name: obra, dtype: int64

In [121]:
df.loc[df["duracion"] == 70, ["obra", "productora", "anio_inicio", "duracion"]]


,obra,productora,anio_inicio,duracion
46,El ascensor,Actor’s First.,2017,70.0


In [84]:
sorted(df["productora"].dropna().unique())


['ATG Entertainment',
 'Actor’s First.',
 'Arabian Horses Production LTD',
 'Beon Entertainment',
 'Beon Entertainment\n',
 'Dagoll Dagom',
 'Drive Entertainment',
 'Euroscena',
 'Ferran González & Alícia Serrat',
 'Hatchwell Inversiones Musicales S.L.',
 "Let's Go Company",
 'Los Javis',
 'Mario Gas',
 'Nostromo Live',
 'Nostromo live',
 'Pentación Espectáculos\n',
 'SMedia',
 'SMedia\n',
 'SOM Produce',
 'Selladoor Worldwide',
 'Smedia',
 'Stage Entertainment',
 'Stage Entertainment\n',
 'Teatro del Soho S.L.',
 'Teatro del Soho S.L.\n',
 'Teatropolis (Gran Teatro CaixaBank Príncipe Pío)',
 'The Stage Company',
 'The William Morris Agency Endeavor Entertainment; Illuminati Production Group; Publintermedia Consulting; JK Productions',
 'Theatre Properties',
 'Theatre Properties\n',
 'Tricicle']

In [122]:
# Normalizar productora
df["productora"] = (
    df["productora"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)  # reemplaza espacios múltiples
    .str.replace("\n", "", regex=False)     # elimina saltos de línea
    .str.strip()                             # elimina espacios extremos
)

# Correcciones conocidas de variaciones
reemplazos = {
    "SMedia": "Smedia",
    "SMedia ": "Smedia",
    "Smedia\n": "Smedia",
    "Beon Entertainment\n": "Beon Entertainment",
    "Stage Entertainment\n": "Stage Entertainment",
    "Teatro del Soho S.L.\n": "Teatro del Soho S.L.",
    "Theatre Properties\n": "Theatre Properties",
}

df["productora"] = df["productora"].replace(reemplazos)

# Mostrar valores únicos ordenados
sorted(df["productora"].dropna().unique())


['ATG Entertainment',
 'Actor’s First.',
 'Arabian Horses Production LTD',
 'Beon Entertainment',
 'Dagoll Dagom',
 'Drive Entertainment',
 'Euroscena',
 'Ferran González & Alícia Serrat',
 'Hatchwell Inversiones Musicales S.L.',
 "Let's Go Company",
 'Los Javis',
 'Mario Gas',
 'Nostromo Live',
 'Nostromo live',
 'Pentación Espectáculos',
 'SOM Produce',
 'Selladoor Worldwide',
 'Smedia',
 'Stage Entertainment',
 'Teatro del Soho S.L.',
 'Teatropolis (Gran Teatro CaixaBank Príncipe Pío)',
 'The Stage Company',
 'The William Morris Agency Endeavor Entertainment; Illuminati Production Group; Publintermedia Consulting; JK Productions',
 'Theatre Properties',
 'Tricicle']

In [123]:
df.loc[
    df["productora"].str.strip().str.lower() == "nostromo live",
    "productora"
] = "Nostromo Live"


In [124]:
df.loc[
    df["productora"].str.contains("som", case=False, na=False),
    "productora"
] = "ATG Entertainment"


In [125]:
sorted(df["productora"].dropna().unique())


['ATG Entertainment',
 'Actor’s First.',
 'Arabian Horses Production LTD',
 'Beon Entertainment',
 'Dagoll Dagom',
 'Drive Entertainment',
 'Euroscena',
 'Ferran González & Alícia Serrat',
 'Hatchwell Inversiones Musicales S.L.',
 "Let's Go Company",
 'Los Javis',
 'Mario Gas',
 'Nostromo Live',
 'Pentación Espectáculos',
 'Selladoor Worldwide',
 'Smedia',
 'Stage Entertainment',
 'Teatro del Soho S.L.',
 'Teatropolis (Gran Teatro CaixaBank Príncipe Pío)',
 'The Stage Company',
 'The William Morris Agency Endeavor Entertainment; Illuminati Production Group; Publintermedia Consulting; JK Productions',
 'Theatre Properties',
 'Tricicle']

In [126]:
df.head(25)


,clave_prod,obra,productora,anio_inicio,anio_fin,teatro,ciudad_principal,gira,fuente_url,activa,duracion
0,"101 dálmatas, el musical | teatropolis (gran t...","101 Dálmatas, el musical",Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2023,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://www.granteatrocaixabankprincipepio.com/,True,85.0
1,aladdín | stage entertainment | 2023 | 2025.0,Aladdín,Stage Entertainment,2023,2025.0,Teatro Coliseum,Madrid,No,https://www.stage.es/musicales/aladdin/,False,145.0
2,anastasia | stage entertainment | 2018 | 2020.0,Anastasia,Stage Entertainment,2018,2020.0,Teatro Coliseum,Madrid,No,https://www.stage.es/musicales/anastasia/,False,145.0
3,avenue q | teatropolis (gran teatro caixabank ...,Avenue Q,Teatropolis (Gran Teatro CaixaBank Príncipe Pío),2024,NaN,Gran Teatro CaixaBank Príncipe Pío,Madrid,No,https://es.wikipedia.org/wiki/Avenue_Q,True,100.0
4,billy elliot | som produce | 2017 | 2020.0,Billy Elliot,ATG Entertainment,2017,2020.0,Nuevo Teatro Alcalá,Madrid,Sí,https://somproduce.com/musicales/billy-elliot/,False,150.0
5,cabaret | let's go company | 2025 | nan,Cabaret,Let's Go Company,2025,NaN,Albéniz,Madrid,No,fuente: observación personal,True,150.0
6,cenicienta | stage entertainment | 2025 | nan,Cenicienta,Stage Entertainment,2025,NaN,Coliseum,Madrid,No,fuente: observación personal,True,145.0
7,charlie y la fábrica de chocolate | let's go c...,Charlie y la fábrica de chocolate,Let's Go Company,2021,2022.0,Espacio Ibercaja Delicias,Madrid,Sí,https://www.teatroalamedia.com/charlie-y-la-fa...,False,150.0
8,chicago | som produce | 2023 | 2025.0,Chicago,ATG Entertainment,2023,2025.0,Teatro Apolo,Madrid,Sí,https://somproduce.com/musicales/chicago/,False,150.0
9,dirty dancing | let's go company | 2018 | 2023.0,Dirty Dancing,Let's Go Company,2018,2023.0,Teatro Nuevo Alcalá,Madrid,Sí,https://www.entradas.com/artist/dirty-dancing/,False,145.0


In [127]:
df.to_csv("maestro_musicales.csv", index=False, encoding="utf-8")
